In [14]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy.spatial.distance import euclidean
from scipy.interpolate import splprep, splev
from shapely.geometry import Polygon
from svgwrite import Drawing

In [2]:
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    
    return path_XYs

In [3]:
def normalize_points(points_list):
    normalized_points_list = []

    for points in points_list:
        min_x, min_y = points.min(axis=0)
        max_x, max_y = points.max(axis=0)
        range_x = max_x - min_x
        range_y = max_y - min_y
        
        range_x = range_x if range_x != 0 else 1
        range_y = range_y if range_y != 0 else 1
        
        normalized_points = (points - [min_x, min_y]) / [range_x, range_y]
        normalized_points_list.append(normalized_points)
    
    return normalized_points_list

In [4]:
def straight_line(points):

    if len(points) < 2:
        return points

    p1, p2 = points[0], points[-1]
    for i in range(1, len(points)):
        slope = (p2[1] - p1[1]) / (p2[0] - p1[0]) if p2[0] != p1[0] else float('inf')
        current_slope = (points[i][1] - p1[1]) / (points[i][0] - p1[0]) if points[i][0] != p1[0] else float('inf')
        if not np.isclose(slope, current_slope, atol=1e-6):
            return points  

    modified_points = []
    for point in points:
        t = np.dot(point - p1, p2 - p1) / np.dot(p2 - p1, p2 - p1)
        modified_points.append(p1 + t * (p2 - p1))
    return np.array(modified_points)

In [5]:
def circle_ellipse(points):
    if len(points) < 5:
        return points

    centroid = np.mean(points, axis=0)
    distances = np.array([euclidean(point, centroid) for point in points])
    mean_distance = np.mean(distances)

    if np.allclose(distances, mean_distance, rtol=1e-2): 
        return np.array([centroid + mean_distance * (point - centroid) / np.linalg.norm(point - centroid) for point in points])
    else:
        a = np.max(distances)
        b = np.min(distances)
        return np.array([centroid + np.array([a * (point[0] - centroid[0]), b * (point[1] - centroid[1])]) / np.linalg.norm(point - centroid) for point in points])

    return points

In [6]:
def rectangle(points):
    if len(points) != 4:
        return points

    rect = sorted(points, key=lambda p: (p[0], p[1]))

    mid1 = (rect[0] + rect[1]) / 2
    mid2 = (rect[2] + rect[3]) / 2

    aligned_points = np.array([
        [mid1[0], rect[0][1]],
        [mid2[0], rect[1][1]],
        [mid2[0], rect[3][1]],
        [mid1[0], rect[2][1]]
    ])

    return aligned_points

In [7]:
def regular_polygon(points):
    n = len(points)
    if n < 3:
        return points

    centroid = np.mean(points, axis=0)

    radius = np.mean([euclidean(point, centroid) for point in points])

    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    modified_points = np.array([
        [centroid[0] + radius * np.cos(angle), centroid[1] + radius * np.sin(angle)]
        for angle in angles
    ])

    return modified_points

In [8]:
def star(points):
    if len(points) < 5:
        return points

    centroid = np.mean(points, axis=0)
    angles = np.linspace(0, 2 * np.pi, len(points), endpoint=False)

    distances = np.linalg.norm(points - centroid, axis=1)
    max_distance = np.max(distances)
    min_distance = np.min(distances)

    if np.abs(max_distance - min_distance) > max_distance * 0.5:
        star_points = []
        for i in range(len(points)):
            angle = angles[i]
            radius = max_distance if i % 2 == 0 else min_distance
            x = centroid[0] + radius * np.cos(angle)
            y = centroid[1] + radius * np.sin(angle)
            star_points.append([x, y])
        return np.array(star_points)

    return points

In [9]:
'''def symmetry(points):
    n = len(points)
    if n < 2:
        return points
    centroid = np.mean(points, axis=0)

    modified_points = np.array([centroid * 2 - point for point in points])

    return modified_points'''
def detect_symmetry(points):
    if len(points) < 2:
        return None
    
    centroid = np.mean(points, axis=0)
    symmetries = []
    
    for i, j in itertools.combinations(range(len(points)), 2):
        midpoint = (points[i] + points[j]) / 2
        if np.allclose(midpoint, centroid, rtol=1e-2):
            symmetries.append((points[i], points[j]))
    
    return symmetries

In [10]:
def complete_curve(points, method='smooth'):
    if len(points) < 3:
        return points
    
    if method == 'smooth':
        tck, _ = splprep(points.T, s=0)
        new_points = splev(np.linspace(0, 1, len(points) * 10), tck)
        return np.array(new_points).T

    elif method == 'regular':
        return np.array(points)
    
    return points

In [11]:
def process_csv_and_output(csv_path, original_svg, modified_svg):
    path_XYs = read_csv(csv_path)

    draw_original = Drawing(original_svg)
    for path in path_XYs:
        for points in path:
            draw_original.add(draw_original.polyline(points, stroke='black', fill='none'))
    draw_original.save()

    normalized_shapes = []
    for path in path_XYs:
        points = straight_line(points)
        points = circle_ellipse(points)
        points = rectangle(points)
        points = regular_polygon(points)
        points = star(points)
        points = complete_curve(points)
        
        normalized_shapes.append(points)

    draw_modified = Drawing(modified_svg)
    for shape in normalized_shapes:
        for points in shape:
            draw_modified.add(draw_modified.polyline(points, stroke='black', fill='none'))
    draw_modified.save()

In [13]:
if __name__ == "__main__":
    input_csv = 'C:/Users/Sohni/OneDrive/Documents/New folder/problems/frag0.csv'
    original_image = 'original_image.svg'
    modified_image = 'modified_image.svg'
    
    process_csv_and_output(input_csv, original_image, modified_image)
    print(f"Original and modified SVG images saved as {original_image} and {modified_image}.")

KeyboardInterrupt: 